In [1]:
import pandas as pd
import xarray as xr
import numpy as np
from glob import glob
from yllib.function import mysom

In [2]:
year1   = 1981
year2   = 2020
season  = 'cold'
months  = [10,11,12,1,2,3]
# season  = 'warm'
# months  = [4,5,6,7,8,9]
mapsize = (4,4)
factor  = ('z')
bmu_filename = f"som/{season}_z_{mapsize[0]}x{mapsize[1]}.nc"
bmu_filename

'som/cold_z_4x4.nc'

In [3]:
data = xr.open_dataset('data/hgt_daily_anomaly.nc')
# select months 
data = data.sel(time=data['time.month'].isin(months))
# select 500-hpa geopotential height for SOM
data = data['z500']
# convert to 1D
data = mysom.convert_to_1D(data)

In [4]:
import sompy
def calc_som(data, mapsize=[8,8]):
    dims = data.shape
    print(data)
    som = sompy.SOMFactory.build(data.data, mapsize, mask=None, mapshape='planar',
                                 lattice='rect', normalization="var",
                                 initialization='pca',
                                 neighborhood='gaussian',
                                 training='batch',
                                 name='sompy')  # this will use the default parameters, but i can change the initialization and neighborhood methods
    som.train(n_job=1, verbose='info', train_rough_len=100, train_finetune_len=50)
    topographic_error = som.calculate_topographic_error()
#    map_optimal_size = som.calculate_map_size(lattice='rect')
#    print(map_optimal_size)
    quantization_error = np.mean(som._bmu[1])
    print(topographic_error)
    print(quantization_error)

    bmu = som._bmu[0]
    bmu = xr.DataArray(bmu, dims=['time'], coords={'time': np.arange(dims[0])}, name='bmu')
    print(bmu)
#        v = sompy.mapview.View2DPacked(*mapsize, 'test',text_size=8)  
#        c1 = som.cluster(n_clusters=4)
#        v.show(som, what="cluster")
#        print(som.cluster_labels)
#        bmu = bmu.assign_attrs({"cluster":som.cluster_labels})
#        h = sompy.hitmap.HitMapView(*mapsize, 'hitmap', text_size=8, show_text=True)
#        h.show(som)

#        u = sompy.umatrix.UMatrixView(100, 100, 'umatrix', show_axis=True, text_size=8, show_text=True)

    #This is the Umat value
#        UMAT  = u.build_u_matrix(som, distance=1, row_normalized=False)

#        UMAT = u.show(som, distance=1, row_normalized=False, show_data=True, contour=True, blob=False)
    return bmu

In [5]:
bmu = calc_som(data, mapsize=mapsize)

<xarray.DataArray 'preSOM' (time: 7290, z: 11651)>
array([[ 307.19569639,  430.7513085 ,  548.52521231, ...,   -3.33405712,
         -11.32420314,  -18.08370517],
       [ -16.50076608,   54.92982404,  142.58657422, ...,    2.32959031,
          -5.90205393,  -10.93550531],
       [ 176.45973612,  240.87277372,  285.4887377 , ...,  -32.81129443,
         -44.14736339,  -52.68890047],
       ...,
       [-170.64117719,  -95.08775797,  -64.20567834, ...,   37.26571836,
          30.8648394 ,   22.22764533],
       [-271.98736096, -128.86410444,   15.60254168, ...,  125.90605032,
         114.32568725,  110.08770486],
       [-365.42012183, -290.67908799, -237.41636479, ...,  325.96106732,
         304.92913245,  284.37740867]])
Coordinates:
  * time       (time) datetime64[ns] 1981-01-01 1981-01-02 ... 2020-12-31
    dayofyear  (time) int64 ...
    longitude  (z) float64 150.0 150.0 150.0 150.0 ... 340.0 340.0 340.0 340.0
    latitude   (z) float64 70.0 69.0 68.0 67.0 66.0 ... 14.0 13.0 

 Training...
 pca_linear_initialization took: 2.354000 seconds
 Rough training...
 radius_ini: 1.000000 , radius_final: 1.000000, trainlen: 100

 epoch: 1 ---> elapsed time:  1.549000, quantization error: 525.954242

 epoch: 2 ---> elapsed time:  1.477000, quantization error: 99.820316

 epoch: 3 ---> elapsed time:  1.440000, quantization error: 97.646188

 epoch: 4 ---> elapsed time:  1.713000, quantization error: 97.432798

 epoch: 5 ---> elapsed time:  1.423000, quantization error: 97.270236

 epoch: 6 ---> elapsed time:  1.350000, quantization error: 97.146465

 epoch: 7 ---> elapsed time:  1.425000, quantization error: 97.046239

 epoch: 8 ---> elapsed time:  1.307000, quantization error: 96.977776

 epoch: 9 ---> elapsed time:  1.342000, quantization error: 96.932353

 epoch: 10 ---> elapsed time:  1.502000, quantization error: 96.898078

 epoch: 11 ---> elapsed time:  1.571000, quantization error: 96.873223

 epoch: 12 ---> elapsed time:  1.639000, quantization error: 96.855880


0.047736625514403296
96.79792072931389
<xarray.DataArray 'bmu' (time: 7290)>
array([3., 3., 3., ..., 2., 7., 2.])
Coordinates:
  * time     (time) int64 0 1 2 3 4 5 6 7 ... 7283 7284 7285 7286 7287 7288 7289


In [6]:
bmu.to_netcdf(bmu_filename)

In [7]:
data

<xarray.DataArray 'preSOM' (time: 7290, z: 11651)>
array([[ 307.19569639,  430.7513085 ,  548.52521231, ...,   -3.33405712,
         -11.32420314,  -18.08370517],
       [ -16.50076608,   54.92982404,  142.58657422, ...,    2.32959031,
          -5.90205393,  -10.93550531],
       [ 176.45973612,  240.87277372,  285.4887377 , ...,  -32.81129443,
         -44.14736339,  -52.68890047],
       ...,
       [-170.64117719,  -95.08775797,  -64.20567834, ...,   37.26571836,
          30.8648394 ,   22.22764533],
       [-271.98736096, -128.86410444,   15.60254168, ...,  125.90605032,
         114.32568725,  110.08770486],
       [-365.42012183, -290.67908799, -237.41636479, ...,  325.96106732,
         304.92913245,  284.37740867]])
Coordinates:
  * time       (time) datetime64[ns] 1981-01-01 1981-01-02 ... 2020-12-31
    dayofyear  (time) int64 ...
    longitude  (z) float64 150.0 150.0 150.0 150.0 ... 340.0 340.0 340.0 340.0
    latitude   (z) float64 70.0 69.0 68.0 67.0 66.0 ... 14.0 13.0 12.0 11.0 10.0
  * z          (z) int64 0 1 2 3 4 5 6 ... 11645 11646 11647 11648 11649 11650